In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd "/content/drive/My Drive/character_recognition"


/content/drive/My Drive/character_recognition


In [ ]:
!ls

dataset_characters.zip	Untitled0.ipynb


In [ ]:
!unzip dataset_characters.zip -d character_dataset

In [ ]:
!touch loading_data.py

In [ ]:
!touch classes.txt

In [ ]:
import numpy as np
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dropout, Dense, Flatten, MaxPooling2D
from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing import image_dataset_from_directory

image_size = (28, 28)
batch_size = 32
train_ds = image_dataset_from_directory(
    "/content/drive/My Drive/character_recognition/character_dataset",
    validation_split=0.2,
    subset="training",
    label_mode="categorical",
    seed=1337,
    color_mode="grayscale",
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = image_dataset_from_directory(
    "/content/drive/My Drive/character_recognition/character_dataset",
    validation_split=0.2,
    subset="validation",
    label_mode="categorical",
    color_mode="grayscale",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 37623 files belonging to 36 classes.
Using 30099 files for training.
Found 37623 files belonging to 36 classes.
Using 7524 files for validation.


In [ ]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(36, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 26, 26, 32)        9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 5, 5, 64)         

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, verbose=1, )

cpt_save = ModelCheckpoint('./weight.h5', save_best_only=True, monitor='val_acc', mode='max')

model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(1e-3), metrics=['acc'])
model.fit(train_ds, callbacks=[cpt_save, reduce_lr],validation_data=val_ds, verbose=1, epochs=50, shuffle=True, batch_size=128)